In [ ]:
from fastai.vision.all import *

In [ ]:
path = Path('/kaggle/input/cassava-leaf-disease-classification/')
path.ls()

In [ ]:
train_df = pd.read_csv(path/'train.csv')
train_df.head()
with open(path/'label_num_to_disease_map.json') as f:
    label_dict = json.load(f)

label_dict = {int(k):v for k,v in label_dict.items()}
df = train_df.set_index('image_id')
labels = df.to_dict()['label']

def get_label(labels, x):
    x = Path(x)
    return labels[x.name]

def get_data(labels, bs=64, presize=500, resize=384):
    tfms = [Rotate(90), Warp(magnitude=0.4, p=1.),Zoom(min_zoom=0.9, max_zoom=1.3), Brightness(max_lighting=0.5), Flip(), Contrast(), Resize(resize)]
    comp = setup_aug_tfms(tfms)
    return DataBlock(blocks=(ImageBlock, CategoryBlock),
        get_items=lambda p: get_image_files(p),
        get_y=partial(get_label, labels),
        splitter=RandomSplitter(),
        item_tfms=[Resize(presize)],
        batch_tfms=[*comp, Normalize.from_stats(*imagenet_stats)]).dataloaders(path/'train_images',bs=bs,num_workers=8)

In [ ]:
dls = get_data(labels,bs=128, presize=384)
learn = cnn_learner(dls, models.resnet50, metrics=[accuracy], pretrained=False)
test_files = get_image_files(path/'test_images')
predictions = []
for fold in range(3):
    learn.load(f'/kaggle/input/resnet50/models/resnet50-full-fold_{fold}')
    test_dl = dls.test_dl(test_files)
    preds = learn.tta(dl=test_dl, n=6)
    predictions.append(preds[0])

In [ ]:
predictions

In [ ]:
preds = torch.argmax(torch.mean(torch.stack(predictions), dim=0), dim=1)
test_fn = map(lambda x: x.name, test_files)

In [ ]:
submission = pd.DataFrame({'image_id': test_fn, 'label': preds})
submission.to_csv('submission.csv', index=False)

In [ ]:
submission